In [ ]:
import numpy as np
import pandas as pd
from scipy import spatial
import volprice as vl
import matplotlib as mpl
import seaborn as sns
import logging
import tqdm
from multiprocessing import Pool
import sys

In [ ]:
%matplotlib inline
mpl.style.use('seaborn-talk')
np.set_printoptions(precision=3)

In [ ]:
logging.captureWarnings(True)
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)
info_handler = logging.FileHandler('../logging.tmp.txt')
info_handler.addFilter(lambda record: record.levelno <= logging.WARNING)
error_handler = logging.StreamHandler(sys.stderr)
error_handler.addFilter(lambda record: record.levelno > logging.WARNING)
root_logger.addHandler(info_handler)
root_logger.addHandler(error_handler)

In [ ]:
vl.__version__

In [ ]:
innov_dim = 2000

## Load the data

In [ ]:
with pd.HDFStore('../data/spy_daily_estimates_2003-2017_1000_1sec.hdf') as store:
    daily_rtn = store['daily_rtn'] * 40
    vol = store['discrete_vol_est'] * 40**2
    
    data = pd.concat([daily_rtn, vol.diffusion + vol.jumps], axis=1).dropna()
    data.columns = ['rtn', 'vol']

In [ ]:
data.head()

In [ ]:
data.transform(lambda x: np.sqrt(x) if x.name == 'vol' else x).cov()

In [ ]:
data.corr()

In [ ]:
data.describe()

In [ ]:
data.describe()

In [ ]:
fig1, ax1 = mpl.pyplot.subplots()
data['rtn'].plot(ax=ax1, color='blue', alpha=.5, label="$r_{t+1}$")
data['vol'].apply(np.sqrt).plot(ax=ax1, label='$\sqrt{\sigma^2_{t+1}}$', color='red')
ax1.legend()
fig1.savefig('../doc/figures/time_series.tmp.pdf', frameon=False, pad_inches=0, bbox_inches='tight',
            transparent=True)

In [ ]:
joint_grid = sns.JointGrid(x='vol', y='rtn', data=data, xlim=[-.1, 9], ylim=[-3.5, 3.5])
joint_grid.plot_joint(sns.regplot, ci=None, color='purple', scatter_kws={"alpha":.5})
sns.kdeplot(data['vol'], ax=joint_grid.ax_marg_x, legend=None, shade=True, color='red')
sns.kdeplot(data['rtn'], ax=joint_grid.ax_marg_y, legend=None, shade=True, color='blue', vertical=True)
joint_grid.set_axis_labels("$\sigma^2_{t+1}$","$r_{t+1}$")
joint_grid.savefig('../doc/figures/joint_dist.tmp.pdf', frameon=False, pad_inches=0, bbox_inches='tight',
            transparent=True),

In [ ]:
omega, omega_cov = vl.estimate_params_strong_id(data)

In [ ]:
estimates, covariance = vl.estimate_params(data, *vl.compute_vol_gmm(data.vol, 
                                                                     vl.compute_init_constants(data.vol))) 
                                           


In [ ]:
estimates

In [ ]:
covariance

In [ ]:
vl.compute_strong_id(estimates, covariance)

In [ ]:
np.exp(1.8030412407446503)

In [ ]:
strong_id_results = vl.estimate_params_strong_id(data) #,  bounds=[(-3, 2), (-1, 3), (0,1)])

In [ ]:
strong_id_results

In [ ]:
# Since log_delta = log_both - log_scale. Asy Var(hat{log_delta}) = var(log_both) + var(log_both)
# - 2 * cov(log_both, log_scale)

In [ ]:
omega

In [ ]:
pd.Series(omega)

In [ ]:
omega_cov

In [ ]:
data.vol.sort_values(ascending=True).head()

In [ ]:
omega2, omega_cov2 = vl.estimate_params(data)

In [ ]:
omega2

In [ ]:
qlr_stats = vl.compute_qlr_stats(omega=omega2, omega_cov=omega_cov2, theta_dim=10, pi_dim=10, phi_dim=10, 
                                 pi_min=-.25, pi_max=0, theta_min=.2, theta_max=.6,  use_tqdm=True)
                               


In [ ]:
qlr_draws = vl.compute_qlr_sim(omega=omega2, omega_cov=omega_cov2, theta_dim=10, pi_dim=10, phi_dim=10, 
                               innov_dim=innov_dim, pi_min=-.25, pi_max=0, theta_min=.2, theta_max=.6,
                               use_tqdm=True, alpha=0.05)

In [ ]:
np.sort(qlr_stats.phi.unique()) - np.sort(qlr_draws.phi.unique())

In [ ]:
qlr_stats.set_index(['phi', 'pi', 'theta']).head()

In [ ]:
qlr_stats.count()

In [ ]:
qlr_stats.sort_values(by=['phi', 'pi', 'theta'], inplace=True)
qlr_draws.sort_values(by=['phi', 'pi', 'theta'], inplace=True)
close_enough = np.allclose(qlr_stats[['phi', 'pi', 'theta']], qlr_draws[['phi', 'pi', 'theta']])

if not close_enough:
    raise RuntimeError('The indices are not the same!!!')
    
else:
    qlr_stats[['phi', 'pi', 'theta']] = qlr_draws[['phi', 'pi', 'theta']]
    
    
merged_values = pd.merge(qlr_stats, qlr_draws,left_on=['phi', 'pi', 'theta'], right_on=['phi', 'pi', 'theta'],
                         suffixes=['_draws', '_stats'])

In [ ]:
qlr_stats.sort_values(by='qlr').head()

In [ ]:
qlr_draws.sort_values(by='qlr').head()

In [ ]:
merged_values.head()

In [ ]:
accepted_vals = merged_values.query('qlr_stats > qlr_draws').drop(['qlr_draws', 'qlr_stats'], axis='columns')

In [ ]:
qlr_draws.to_json(f'../results/qlr_draws_on_data_{innov_dim}_smaller_region.json')

In [ ]:
qlr_draws = pd.read_json(f'../results/qlr_draws_on_data_{innov_dim}_smaller_region.json')

In [ ]:
def constraint(scale, psi, zeta, equity_price):
    
    vol_price  = -1 / scale - ((psi * (equity_price-1) + zeta/2 * (equity_price-1)**2))
    
    return  vol_price

In [ ]:
accepted_vals

In [ ]:
omega

In [ ]:
fig3, ax3 = mpl.pyplot.subplots()
ax3.scatter(y=accepted_vals.theta, x=accepted_vals.pi, marker='o', s=650, label='Not Rejected Values',
           color='purple')
ax3.set_xlim([-3.5, .5])
ax3.set_ylim([-.5, 3.5])
yvals = np.linspace(0,1,100)
# xvals = [constraint(np.exp(omega['log_scale']), omega['psi'], omega['zeta'], x) for x in yvals]
# ax3.plot(xvals, yvals, color='black', label='Constraint')
ax3.axvline(0,  ymin=.5 /6, ymax=1 - .5/6, color='black')
ax3.axhline(0,  xmin=.915, xmax=0, color='black')

white_rect = mpl.patches.Rectangle((0,0), .1, 1, angle=0.0, color='white', fill=True)
ax3.add_patch(white_rect)

ax3.set_ylabel(r'$\theta$')
ax3.set_xlabel(r'$\pi$')
ax3.set_ylim([-0.1, 1.1])
ax3.set_xlim([-1.1, 0.1])
ax3.legend(loc='upper left')

fig3.savefig('../doc/figures/confidence_region_2.tmp.pdf', frameon=False, pad_inches=0, bbox_inches='tight',
            transparent=True)

In [ ]:
accepted_vals.min()

In [ ]:
accepted_vals.max()